In [1]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt

from tslearn.utils import to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

In [2]:
# helper functions

def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]

def series_to_arr(series):
    l = []
    for key in series:
        if (key[-1] == 'v'): continue
        l += (series[key])
    return l

def E3(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 3):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E2(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 2):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E1(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 1):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E0(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) == 0):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

In [4]:
# loading train and test sets

raw_X_train = []
raw_X_test = []
smoothed_X_train = []
smoothed_X_test = []
translate_X_train = []
translate_X_test = []
final_X_train = []
final_X_test = []

o_label_train = []
o_label_test = []

f_label_train = []
f_label_test = []

b_label_train = []
b_label_test = []

s_label_train = []
s_label_test = []

l_label_train = []
l_label_test = []

with open("..\\training_examples.txt") as t:
    training_examples = t.readlines()

with open("..\\test_examples.txt") as t:
    test_examples = t.readlines()

for file in training_examples:
    raw_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\1_unprocessed\\' + file[:-1])
    smoothed_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\2_smoothed\\' + file[:-1])
    translate_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\3_translation\\' + file[:-1])
    final_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\4_final\\' + file[:-1])

    raw_X_train.append(series_to_arr(raw_series))
    smoothed_X_train.append(series_to_arr(smoothed_series))
    translate_X_train.append(series_to_arr(translate_series))
    final_X_train.append(series_to_arr(final_series))

    o_label_train.append(int(file[4]))
    f_label_train.append(int(file[7]))
    b_label_train.append(int(file[10]))
    s_label_train.append(int(file[13]))
    l_label_train.append(int(file[16]))

for file in test_examples:
    raw_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\1_unprocessed\\' + file[:-1])
    smoothed_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\2_smoothed\\' + file[:-1])
    translate_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\3_translation\\' + file[:-1])
    final_series, num_frames = load_series('..\\time_series\\Time_normalized_stages\\4_final\\' + file[:-1])

    raw_X_test.append(series_to_arr(raw_series))
    smoothed_X_test.append(series_to_arr(smoothed_series))
    translate_X_test.append(series_to_arr(translate_series))
    final_X_test.append(series_to_arr(final_series))

    o_label_test.append(int(file[4]))
    f_label_test.append(int(file[7]))
    b_label_test.append(int(file[10]))
    s_label_test.append(int(file[13]))
    l_label_test.append(int(file[16]))

raw_X_train = to_time_series_dataset(raw_X_train)
raw_X_test = to_time_series_dataset(raw_X_test)

smoothed_X_train = to_time_series_dataset(smoothed_X_train)
smoothed_X_test = to_time_series_dataset(smoothed_X_test)

translate_X_train = to_time_series_dataset(translate_X_train)
translate_X_test = to_time_series_dataset(translate_X_test)

final_X_train = to_time_series_dataset(final_X_train)
final_X_test = to_time_series_dataset(final_X_test)

print(l_label_train)

[1, 1, 7, 7, 8, 2, 2, 2, 2, 7, 7, 7, 8, 8, 9, 3, 3, 3, 7, 7, 2, 1, 5, 7, 7, 4, 4, 9, 9, 5, 5, 2, 2, 7, 7, 6, 6, 2, 2, 4, 4, 3, 7, 7, 6, 6, 4, 4, 2, 2, 6, 6, 6, 6, 7, 7, 2, 2, 6, 6, 3, 3, 2, 9, 4, 2, 2, 9, 9, 1, 5, 5, 4, 4, 7, 7, 5, 5, 7, 5, 5, 6, 6, 6, 6, 6, 6, 9, 9, 2, 4, 4, 7, 0, 0, 0, 0, 0, 1, 1, 0, 7, 7, 7, 7, 5, 5, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 8, 0, 0, 0, 0, 1, 1, 3, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 3, 1, 1, 2, 2, 1, 1, 1, 1, 0, 0, 5, 5, 0, 0, 3, 5, 4, 4, 0, 0, 3, 3, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 0, 3, 1, 3, 3, 0, 0, 1, 1, 0, 1, 6, 6, 0, 1, 1, 0, 0, 0, 0, 1, 1, 3, 3, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 7, 7, 1, 1, 3, 3, 0, 1, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 1, 1, 6, 0, 0, 4, 4, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 6, 6, 0, 0, 6, 0, 0, 

In [5]:
# checking accuracies for KNN
oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
oknn.fit(raw_X_train, o_label_train)
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
fknn.fit(raw_X_train, f_label_train)
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
bknn.fit(raw_X_train, b_label_train)
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
sknn.fit(raw_X_train, s_label_train)
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9, metric='euclidean')
lknn.fit(raw_X_train, l_label_train)

oknn_pred = oknn.predict(raw_X_test)
fknn_pred = fknn.predict(raw_X_test)
bknn_pred = bknn.predict(raw_X_test)
sknn_pred = sknn.predict(raw_X_test)
lknn_pred = lknn.predict(raw_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))


print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9, metric='euclidean')

oknn.fit(smoothed_X_train, o_label_train)
fknn.fit(smoothed_X_train, f_label_train)
bknn.fit(smoothed_X_train, b_label_train)
sknn.fit(smoothed_X_train, s_label_train)
lknn.fit(smoothed_X_train, l_label_train)

oknn_pred = oknn.predict(smoothed_X_test)
fknn_pred = fknn.predict(smoothed_X_test)
bknn_pred = bknn.predict(smoothed_X_test)
sknn_pred = sknn.predict(smoothed_X_test)
lknn_pred = lknn.predict(smoothed_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))



print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9, metric='euclidean')

oknn.fit(translate_X_train, o_label_train)
fknn.fit(translate_X_train, f_label_train)
bknn.fit(translate_X_train, b_label_train)
sknn.fit(translate_X_train, s_label_train)
lknn.fit(translate_X_train, l_label_train)

oknn_pred = oknn.predict(translate_X_test)
fknn_pred = fknn.predict(translate_X_test)
bknn_pred = bknn.predict(translate_X_test)
sknn_pred = sknn.predict(translate_X_test)
lknn_pred = lknn.predict(translate_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))




print('\n\n\n\n')

oknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
fknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
bknn = KNeighborsTimeSeriesClassifier(n_neighbors=4, metric='euclidean')
sknn = KNeighborsTimeSeriesClassifier(n_neighbors=6, metric='euclidean')
lknn = KNeighborsTimeSeriesClassifier(n_neighbors=9, metric='euclidean')

oknn.fit(final_X_train, o_label_train)
fknn.fit(final_X_train, f_label_train)
bknn.fit(final_X_train, b_label_train)
sknn.fit(final_X_train, s_label_train)
lknn.fit(final_X_train, l_label_train)

oknn_pred = oknn.predict(final_X_test)
fknn_pred = fknn.predict(final_X_test)
bknn_pred = bknn.predict(final_X_test)
sknn_pred = sknn.predict(final_X_test)
lknn_pred = lknn.predict(final_X_test)

print(str(E0(oknn_pred, o_label_test)))
print(str(E1(oknn_pred, o_label_test)))
print(str(E2(oknn_pred, o_label_test)))
print(str(E3(oknn_pred, o_label_test)))

print(str(E0(fknn_pred, f_label_test)))
print(str(E1(fknn_pred, f_label_test)))
print(str(E2(fknn_pred, f_label_test)))
print(str(E3(fknn_pred, f_label_test)))

print(str(E0(bknn_pred, b_label_test)))
print(str(E1(bknn_pred, b_label_test)))
print(str(E2(bknn_pred, b_label_test)))
print(str(E3(bknn_pred, b_label_test)))

print(str(E0(sknn_pred, s_label_test)))
print(str(E1(sknn_pred, s_label_test)))
print(str(E2(sknn_pred, s_label_test)))
print(str(E3(sknn_pred, s_label_test)))

print(str(E0(lknn_pred, l_label_test)))
print(str(E1(lknn_pred, l_label_test)))
print(str(E2(lknn_pred, l_label_test)))
print(str(E3(lknn_pred, l_label_test)))

26.881720430107528
56.98924731182796
70.96774193548387
80.64516129032258
65.59139784946237
72.04301075268818
74.19354838709677
83.87096774193549
35.483870967741936
46.236559139784944
52.68817204301075
65.59139784946237
21.50537634408602
48.38709677419355
66.66666666666667
77.41935483870968
48.38709677419355
65.59139784946237
77.41935483870968
87.09677419354838





26.881720430107528
59.13978494623656
73.11827956989248
80.64516129032258
67.74193548387096
73.11827956989248
75.26881720430107
83.87096774193549
37.634408602150536
48.38709677419355
55.913978494623656
67.74193548387096
19.35483870967742
55.913978494623656
68.81720430107526
76.34408602150538
49.46236559139785
65.59139784946237
77.41935483870968
87.09677419354838





21.50537634408602
53.763440860215056
66.66666666666667
80.64516129032258
62.365591397849464
69.89247311827957
76.34408602150538
86.02150537634408
33.333333333333336
49.46236559139785
56.98924731182796
65.59139784946237
19.35483870967742
50.53763440860215
72.04301